# Analyze metadata semantically with an LLM

In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 500
pd.options.display.max_seq_items = 500

from datetime import datetime
from tqdm.notebook import tqdm
import requests
from concurrent.futures import ThreadPoolExecutor
from utils import do_full_analysis, parse_analysis_results
import warnings

warnings.simplefilter("ignore", category=(UserWarning, FutureWarning))

In [3]:
# Constants
# Dataset links are composed of this baselink and the identifier for each dataset.
BASELINK_DATASHOP = (
    "https://www.zh.ch/de/politik-staat/statistik-daten/datenkatalog.html#/datasets/"
)

MDV_DATA_PATH = "_data/01_mdv_metadata.parq"

# Default figure size for Quarto HTML output.
FIGSIZE = (7, 5)

In this notebook we perform a specific semantic analysis of titles and descriptions in our metadata catalog with LLMs.

## Load data

**Retrieve data from the metadata API.**

In [4]:
# Retrieve metadata for all datasets from MDV API.
raw = requests.get("https://www.web.statistik.zh.ch/ogd/daten/zhweb.json").json()[
    "dataset"
]
df = pd.DataFrame(pd.json_normalize(raw))
df.to_parquet(MDV_DATA_PATH)

In [5]:
df = pd.read_parquet(MDV_DATA_PATH)
df["publisher"] = df.identifier.str.split("@").str[1]

# Only keep actual OGD datasets. Filter out studies.
is_ogd = df.dropna(subset=["keyword"]).keyword.apply(lambda x: "ogd" in x)
df = df.loc[is_ogd[is_ogd == True].index]
df.reset_index(drop=True, inplace=True)
if len(df) == 0:
    raise ValueError("No data retrieved.")
else:
    print(f"Retrieved {len(df)} datasets.")

Retrieved 445 datasets.


## Analyze and score metadata

In [6]:
# Processing the 445 datasets takes about 14 minutes and costs around 5 CHF.

# Be aware not to hit your OpenAI API rate limits with to many parallel requests.
n_parallel = 10

# Create a list of data rows to process in parallel.
data_rows = [x[1] for x in list(df.iterrows())]

with ThreadPoolExecutor(max_workers=n_parallel) as executor:
    results = list(executor.map(do_full_analysis, data_rows))

In [8]:
results_parsed = pd.DataFrame(results, columns=["results_raw"])
results_parsed = results_parsed["results_raw"].apply(parse_analysis_results)
results_parsed = pd.concat(results_parsed.tolist(), axis=0)
results_parsed.reset_index(drop=True, inplace=True)
df_final = pd.concat([df, results_parsed], axis=1)

cols = [
    "identifier",
    "publisher",
    "title",
    "description",
    "content_score",
    "context_score",
    "quality_score",
    "spacial_score",
    "content",
    "context",
    "quality",
    "spacial",
]

timestamp = datetime.now().strftime("%Y%m%d")
df_final[cols].to_excel(f"_results/metadata_analysis_{timestamp}.xlsx", index=False)